### Import libraries

In [43]:
import sys
import os
sys.path.insert(0, os.path.abspath('../'))

from src.classification import *

In [44]:
pred_classes = ['Customers', 'Attrited']

### Load data

In [45]:
# Load the data
df = pd.read_csv("../data/BankChurners_CleanedMapped.csv", index_col="ClientID")
df.shape

(10127, 20)

In [46]:
column_names = list(df.columns)
#column_names

####  Prepare data
##### Split data to train/test, features/target

In [47]:
X_train, X_test, y_train, y_test = clean_split_data(df, column_names)

#####  Scale features

In [48]:
X_train, X_test = scale_data(X_train, X_test)

### Logistic Regression Classifier

In [51]:
lrClassifier, report = fit_and_test_LR_classifier(X_train, X_test, y_train, y_test, pred_classes, ros=False)
print("\n", report)

Train:       0.9086532526848538
Test:        0.8928923988153998

               precision    recall  f1-score   support

   Customers       0.91      0.96      0.94      1700
    Attrited       0.74      0.52      0.61       326

    accuracy                           0.89      2026
   macro avg       0.83      0.74      0.77      2026
weighted avg       0.88      0.89      0.89      2026



#### LR Classifier w/ OverSampling

In [52]:
lrClassifier_ros, report = fit_and_test_LR_classifier(X_train, X_test, y_train, y_test, pred_classes, ros=True)
print("\n", report)

Train(ros):  0.8522794117647059
Train:       0.8503888408838415
Test:        0.8509378084896347

               precision    recall  f1-score   support

   Customers       0.96      0.86      0.91      1700
    Attrited       0.52      0.81      0.64       326

    accuracy                           0.85      2026
   macro avg       0.74      0.83      0.77      2026
weighted avg       0.89      0.85      0.86      2026



### Random Forest Classifier

In [53]:
rfClassifier, report = fit_and_test_RF_classifier(X_train, X_test, y_train, y_test, pred_classes)
print("\n", report)

Train:       0.9222318232316998
Test:        0.9146100691016782

               precision    recall  f1-score   support

   Customers       0.92      0.98      0.95      1700
    Attrited       0.86      0.56      0.68       326

    accuracy                           0.91      2026
   macro avg       0.89      0.77      0.81      2026
weighted avg       0.91      0.91      0.91      2026



#### RF Classifier w/ OverSampling

In [54]:
rfClassifier_ros, report = fit_and_test_RF_classifier(X_train, X_test, y_train, y_test, pred_classes, ros=True)
print("\n", report)

Train(ros):  0.9199264705882353
Train:       0.9065547463276139
Test:        0.8923988153998026

               precision    recall  f1-score   support

   Customers       0.97      0.90      0.93      1700
    Attrited       0.62      0.88      0.72       326

    accuracy                           0.89      2026
   macro avg       0.80      0.89      0.83      2026
weighted avg       0.92      0.89      0.90      2026



#### RF Classifier w/ balancing parameter

In [55]:
rfClassifier_rfb, report = fit_and_test_RF_classifier(X_train, X_test, y_train, y_test, pred_classes, rfb=True)
print("\n", report)

Train:       0.9233427971855327
Test:        0.9062191510365252

               precision    recall  f1-score   support

   Customers       0.97      0.91      0.94      1700
    Attrited       0.66      0.87      0.75       326

    accuracy                           0.91      2026
   macro avg       0.82      0.89      0.85      2026
weighted avg       0.92      0.91      0.91      2026



##### ... extract feature importance and trim feature list

In [56]:
importances = rfClassifier_rfb.feature_importances_
df_feature_importance = pd.DataFrame(
        list(zip(column_names[1:], list(importances))), 
        columns=["Feature", "Score"]
    )
display(df_feature_importance.sort_values("Score", ascending=False))
df_feature_reduced = df_feature_importance[df_feature_importance["Score"] >= 0.02]
important_features = list(df_feature_reduced["Feature"])
display(important_features)

,Feature,Score
16,TotalTransactionCount_Last12Months,0.304737
15,TotalTransactionAmount_Last12Months,0.186734
12,TotalRevolvingBalance_CC,0.145714
18,AveCardUtilizationRatio,0.089375
17,TransactionCount_ChangeQ4overQ1,0.088953
14,TransactionAmount_ChangeQ4overQ1,0.053934
8,TotalProductNo,0.039031
9,MonthsInactive_Last12Months,0.023668
10,ContactNo_Last12Months,0.020910
11,CreditLimit,0.014159


['TotalProductNo',
 'MonthsInactive_Last12Months',
 'ContactNo_Last12Months',
 'TotalRevolvingBalance_CC',
 'TransactionAmount_ChangeQ4overQ1',
 'TotalTransactionAmount_Last12Months',
 'TotalTransactionCount_Last12Months',
 'TransactionCount_ChangeQ4overQ1',
 'AveCardUtilizationRatio']

In [57]:
column_names_important_features = ["Attrited"] + important_features

### Retrain Random Forest w/ only Important Features 

In [58]:
X_train, X_test, y_train, y_test = clean_split_data(df, column_names_important_features)
X_train, X_test = scale_data(X_train, X_test)

In [59]:
rfClassifier_RFW = RFC(
    n_estimators=100, max_depth=5, bootstrap=True, class_weight="balanced", random_state=1
).fit(X_train, y_train)

print("Train: ", rfClassifier_RFW.score(X_train, y_train))
print("Test:  ", rfClassifier_RFW.score(X_test, y_test))

y_predict_w = rfClassifier_RFW.predict(X_test)

report_w = CR(y_test, y_predict_w, target_names=pred_classes)
print("\n", report_w)

Train:  0.9300086409085299
Test:   0.9131293188548865

               precision    recall  f1-score   support

   Customers       0.98      0.92      0.95      1700
    Attrited       0.67      0.90      0.77       326

    accuracy                           0.91      2026
   macro avg       0.83      0.91      0.86      2026
weighted avg       0.93      0.91      0.92      2026



In [ ]:
#importances = rfClassifier_RFW.feature_importances_
#df_feature_importance = pd.DataFrame(
#        list(zip(column_names_important_features[1:], list(importances))), 
#        columns=["Feature", "Score"]
#    )
#display(df_feature_importance.sort_values("Score", ascending=False))